In [ ]:
# default_exp preprocessing

In [33]:
#export

import srsly
from ssda_nlp.xml_parser import *
import pandas as pd

In [34]:
#export

def xml_to_jsonl(path_to_xml_transcription):
    xml_transcription = open(path_to_xml_transcription, 'r', encoding="utf-8")
    prodigy_input = open(path_to_xml_transcription[:path_to_xml_transcription.find(".xml")] + ".jsonl", 'w', encoding="utf-8")
    
    in_entry = False
    current_entry = ''
    
    for line in xml_transcription:
        if "<entry>" in line:
            in_entry = True            
        elif in_entry and ("</entry>" in line):
            current_entry += line[:line.find("</entry>")]
            in_entry = False            
            prodigy_input.write("{\"text\":\"" + current_entry + "\"}\n")
            current_entry = ''
        elif in_entry:
            if ((line[len(line) - 1] == '\n') or (line[len(line) - 1] == '\r')) and (line[len(line) - 2] == '-'):
                current_entry += line[:len(line) - 2]
            elif line[len(line) - 1] == '-':
                current_entry += line[:len(line) - 1]
            elif (line[len(line) - 1] == '\n') or (line[len(line) - 1] == '\r'):
                current_entry += line[:len(line) - 1] + ' '
            else:
                current_entry += line
            
    xml_transcription.close()
    prodigy_input.close()
    
    return

In [35]:
#no_test

xml_to_jsonl("brazilsourcesvalid.xml")

In [36]:
#export

def parse_annotation(path_to_annotation):   
    annotation = srsly.read_jsonl(path_to_annotation)
    
    spans = []
    texts = []
    
    for entry in annotation:        
        texts.append(entry["text"])
        temp = []
        if "spans" in entry:            
            for span in entry["spans"]:
                temp.append([span["start"], span["end"], span["label"]])
        spans.append(temp)
    
    #build list of unique entries and list of empty annotation dictionaries for each    
    annot_ls = []
    
    for text in texts:        
        annot_ls.append({"entities":[]})
            
    #populate annotation dictionaries
    for i in range(len(texts)):
        for span in spans[i]:        
            annot_ls[i]["entities"].append((int(span[0]), int(span[1]), span[2]))       
        
    #build list of tuples
    tuples = []    
    for i in range(len(texts)):
        tuples.append((texts[i], annot_ls[i]))
        
    return tuples       

In [37]:
#no_test

print(parse_annotation("cabofrio_brazil_1675_1717.annotations.jsonl")[0])

('[margin]: Balthezar.Aos Aos treze do mes de janeiro da era de mil e seis centos e setenta e seis baptizei e pusos s.tos oleos a B.ar filho de Manoel Gonsalves e de sua m.er Domingas de Almeida forão padrinhos o Capitam Bartholomeu [] e Francisca de Magalhães mulher de Antonio.[signed]: Vigario Padre João Pereira. ', {'entities': [(10, 19, 'PER'), (24, 33, 'DATE'), (37, 51, 'DATE'), (52, 96, 'DATE'), (128, 132, 'PER'), (133, 138, 'REL'), (142, 158, 'PER'), (168, 172, 'REL'), (173, 192, 'PER'), (199, 208, 'REL'), (211, 218, 'CHAR'), (219, 230, 'PER'), (236, 258, 'PER'), (259, 265, 'REL'), (269, 284, 'PER'), (287, 300, 'CHAR'), (301, 313, 'PER')]})


In [38]:
#export

def prodigy_output_to_collated_df(path_to_annotation):
    tuples = parse_annotation(path_to_annotation)
    entry_nos = []
    entry_texts = []
    entities = []
    starts = []
    ends = []
    labels = []
    entry = 1
    for tup in tuples:
        for entity in tup[1]["entities"]:
            entry_nos.append(entry)
            entry_texts.append(tup[0])
            entities.append(tup[0][entity[0]:entity[1]])
            starts.append(entity[0])
            ends.append(entity[1])
            labels.append(entity[2])
        entry += 1
    
    collated_dict = {"entry_no": entry_nos, "text": entry_texts, "entity": entities, "start": starts, "end": ends, "label": labels}
    
    collated_df = pd.DataFrame(collated_dict)    
    
    return collated_df

In [39]:
#no_test

mat_xml_df = prodigy_output_to_collated_df("cabofrio_brazil_1675_1717.annotations.jsonl")
mat_xml_df.head()

,entry_no,text,entity,start,end,label
0,1,[margin]: Balthezar.Aos Aos treze do mes de ja...,Balthezar,10,19,PER
1,1,[margin]: Balthezar.Aos Aos treze do mes de ja...,Aos treze,24,33,DATE
2,1,[margin]: Balthezar.Aos Aos treze do mes de ja...,mes de janeiro,37,51,DATE
3,1,[margin]: Balthezar.Aos Aos treze do mes de ja...,da era de mil e seis centos e setenta e seis,52,96,DATE
4,1,[margin]: Balthezar.Aos Aos treze do mes de ja...,B.ar,128,132,PER


In [40]:
#no_test

from nbdev.export import notebook2script
notebook2script()

ModuleNotFoundError: No module named '_nbdev'